### Let"s create a visualisation for the groupschat of a field hockey team :)

Import the used packages.

In [ ]:
from pathlib import Path
from loguru import logger
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tomllib
import numpy as np
import openpyxl


Use the same way as the notebook in the course folder to get the data.

In [ ]:
configfile = Path("../config.toml").resolve()
with configfile.open("rb") as f:
    config = tomllib.load(f)
config

In [ ]:
root = Path("..").resolve()
processed = root / Path(config["processed"])
raw = root / Path(config["raw"])
datafile = processed / config["wife_file"]
role_file = raw / config["role_file"]


In [ ]:
merged_df = pd.read_parquet(datafile)
merged_df.dtypes
merged_df["message_length"] = merged_df["message"].str.len()
merged_df.head()

In [ ]:
# Voeg een kolom toe voor de weekdag
df= merged_df
df['weekday'] = df['timestamp'].dt.day_name()

# Groepeer op weekdag en tel het aantal berichten
message_counts = df.groupby('weekday').size()

# Sorteer de weekdagen in de juiste volgorde
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
message_counts = message_counts.reindex(ordered_days)

# Maak een lijngrafiek van de verdeling van het aantal berichten per weekdag
plt.figure(figsize=(10, 6))
message_counts.plot(kind='line', marker='o')
plt.title('Distribution of Number of Messages per Weekday')
plt.xlabel('Weekday')
plt.ylabel('Number of Messages')
plt.grid(True)
plt.show()

In [ ]:

df = merged_df
message_counts = df.groupby('weekday').size()
message_counts = message_counts.reindex(ordered_days)
# Voeg een kolom toe voor de weekdag
df['weekday'] = df['timestamp'].dt.day_name()

# Zet de 'weekday' kolom om naar een categorische kolom met de juiste volgorde
weekdays_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['weekday'] = pd.Categorical(df['weekday'], categories=weekdays_order, ordered=True)

# # Telling van het aantal berichten per weekdag
# message_counts = df.groupby('weekday').size()

# Maak een figure en assen voor de boxplot en de telling
fig, ax1 = plt.subplots(figsize=(12, 8))

# Maak de boxplot visualisatie per weekdag voor de message_length van de berichten
boxprops = dict(linestyle='-', linewidth=2)
medianprops = dict(linestyle='-', linewidth=2)
flierprops = dict(marker='o', color='red', markersize=5)

# Maak de boxplot en zet de 'weekday' als de x-as
df.boxplot(column='message_length', by='weekday', boxprops=boxprops, medianprops=medianprops, flierprops=flierprops, showfliers=False, ax=ax1)

# Voeg de lijn toe voor het aantal berichten per weekdag
ax2 = ax1.twinx()  # Maak een tweede y-as
ax2.plot(message_counts[weekdays_order].values, color='black', marker='o', linestyle='-', linewidth=3, markersize=7, label='Aantal berichten')

# Stel labels en titels in
# ax2.set_ylabel('Aantal berichten', color='orange')
# ax2.tick_params(axis='y', labelcolor='orange')
plt.title('Boxplot of Message Length by Weekday with Number of Messages')
plt.suptitle('')
plt.xlabel('Weekday')
ax1.set_ylabel('Message Length')
ax1.grid(False)

# Toon de plot
plt.show()


In [ ]:
# Voeg een kolom toe voor de maand en het jaar
df['month_year'] = df['timestamp'].dt.to_period('M')

# Maak een boxplot visualisatie per maand voor de message_length_average van de berichten
plt.figure(figsize=(12, 8))
boxprops = dict(linestyle='-', linewidth=2)
medianprops = dict(linestyle='-', linewidth=2)
flierprops = dict(marker='o', color='red', markersize=5)
df.boxplot(column='message_length_average', by='month_year', boxprops=boxprops, medianprops=medianprops, flierprops=flierprops, showfliers=False)
plt.title('Boxplot of Message Length Average by Month')
plt.suptitle('')
plt.xlabel('Month')
plt.ylabel('Message Length Average')
plt.grid(False)

# Telling van het aantal berichten per maand
message_counts = df.groupby('month_year').size()

# Voeg message counts toe onder de plot
plt.figtext(0.5, -0.05, f'Message Counts per Month:\n{message_counts.to_string()}', ha='center')

plt.show()

In [ ]:
df = merged_df

# Voeg een kolom toe voor de weekdag en het tijdstip
df['weekday'] = df['timestamp'].dt.day_name()
df['time'] = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60

# Sorteer de weekdagen in de juiste volgorde
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['weekday'] = pd.Categorical(df['weekday'], categories=ordered_days, ordered=True)

# Maak een scatter plot van de weekdag en het tijdstip
plt.figure(figsize=(10, 6))
plt.scatter(df['time'], df['weekday'])
plt.title('Weekday vs Time')
plt.xlabel('Time (hours)')
plt.ylabel('Weekday')
plt.grid(True)
plt.show()

In [ ]:
merged_df["message"] = merged_df["message"].fillna("")
kids = r"(?i)lot|levi|lot esther|lot esther stolk|cornelisz|yannick|hoogland|amersfoort|cothen|billie|18 november|17 november|17-11.*|18-11.*|geboorte|geboortezaak|leonie|verloskundige|bevalling"
sports = r"(?i)squash.*|hockey.*|hardlopen.*|hardloop|voetbal|formule 1|verstappen|voordaan|\uD83C\uDFD1|training|club|wedstrijd|gewonnen|verloren|oefenwedstrijd|wielrennen|poolen|fietsen|gelijkspel|0-0|1-0|2-0|1-1|verliezen|verzamelen|uitwedstrijd|thuiswedstrijd|groenekan|de bilt|nova|korfbal"
liefde = r"(?i)liefde|hou van jou|love you|denk aan je|\u2764\uFE0F|\uD83D\uDE0D|\uD83D\uDC98|\uD83D\uDC9D|\uD83D\uDC95|\uD83D\uDC96|\uD83D\uDC97|\uD83D\uDC93|\uD83D\uDC9E|\uD83D\uDC9F|\uD83D\uDC8B|\uD83D\uDC91|\uD83D\uDC8F|\uD83D\uDC8C"
koosnaampje = r"(?i)schat|lief|liefje|schatje|mop.*|lieverd|mooierd|knapperd|lekker ding|liefste"
vrienden = r"(?i) nick|daan|mieke|sandra|yoni|suus|suzanne|coen|patrick|jeroen|emiel|sascha|hans|hidde|bob|marin"
family = r"(?i) maarten|fre|frederike|pieter|reinier|luuk|erna|moeder|vader|ronald|joris|thieme|hidde|ryan|esther|zus|broer|kyano|jay|jayvano|genayro|anne"
animals = r"(?i) hond|billie|billsky|uitlaten|hector|paard|kip|poes|kat|koe|bertha|muis|toe maar|brokken|honden.*|bench|mand|welkoop"
day_to_day = r"(?i) eten we vanavond|supermarkt|jumbo|appie|albert heyn|lidl|ophalen|werken|werk|bij jou?|winkel"
questions = r".*\?$"
leisure = r"(?i)uit eten|bioscoop|film|serie|game of thrones|breaking bad|b&b*|vakantie|weekendje weg|terrasje|drankje|bankhangen|bank hangen|spelletje|rummikub|xbox|gamen|cod|bbq|barbecue|serie|mocro maffia|first dates|winter vol liefde|scrabble"
huwelijk = r"(?i)huwelijk.*|trouwen|7 juni|draaiboek|ceremonie.*|ringen|trouw.*|gastenlijst|weggeven|07-06.*|I do|Jawoord|ten huwelijk|hand vragen|ja-woord|Krista|charlotte|kasteel.*|bruids.*|jurk|pak|dj|rutger|cocktailbar"
werk = r"(?i)ordina|sopra.*|steria|gemeente|gemeentehuis|gemeentewijkbij.*|burgemeester|iris|ilse|collega.*|sandra|mariska|madeloes|jan|wethouder|dashboard|rapportage|werk|werken|reporting|salaris|loon|vakantiedagen|NS|duravermeer|dura|emiel|dave|davey|jasper|annemiek|laura|dennis"
home = r"(?i)verbouwen|zolder|tuin|zwembad|uitbouw|dakkapel|Beatrix|Beatrixstraat.*|Bertha|Bloemenwaard|cothen|huis kopen|woonkamer|hue|uitbouw|zolder|slaapkamer|schuifpui|deur|camera.*|unifi|deurbel|voortuin|schuur|douche|badkamer|verhuizen"
short = r"(?i)^(yes|ja|no|nee|ok|oke|okee|ja ja|nee nee)$"

merged_df["Kinderen"] = merged_df["message"].str.contains(kids, case=False).astype(int)
merged_df["Liefde"] = merged_df["message"].str.contains(liefde, case=False).astype(int)
merged_df["Sport"] = merged_df["message"].str.contains(sports, case=False).astype(int)
merged_df["Vrienden"] = merged_df["message"].str.contains(vrienden, case=False).astype(int)
merged_df["Familie"] = merged_df["message"].str.contains(family, case=False).astype(int)
merged_df["Dieren"] = merged_df["message"].str.contains(animals, case=False).astype(int)
merged_df["Dagelijks"] = merged_df["message"].str.contains(day_to_day, case=False).astype(int)
merged_df["Vrije tijd"] = merged_df["message"].str.contains(leisure, case=False).astype(int)
merged_df["Huwelijk"] = merged_df["message"].str.contains(huwelijk, case=False).astype(int)
merged_df["Werk"] = merged_df["message"].str.contains(werk, case=False).astype(int)
merged_df["Huis"] = merged_df["message"].str.contains(home, case=False).astype(int)
merged_df["topic_short"] = merged_df["message"].str.contains(short, case=False).astype(int)
merged_df["topic_sweet"] = merged_df["message"].str.contains(koosnaampje, case=False).astype(int)

new_columns_count = merged_df[['Kinderen', 'Liefde', 'Sport', 'Vrienden', 'Familie', 'Dieren', 'Dagelijks', 'Vrije tijd', 'Huwelijk', 'Werk', 'Huis']].sum()
merged_df.head()
print(new_columns_count)

In [ ]:
sorted_columns_count = new_columns_count.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sorted_columns_count.plot(kind='bar')
plt.title('Counts of Topics')
plt.xlabel('Topics')
plt.ylabel('Counts')
plt.xticks(rotation=45)
plt.show()

In [ ]:
demographic = merged_df[['author', 'topic_children', 'topic_love', 'topic_sports', 'topic_friends', 'topic_family',
                         'topic_animals', 'topic_day_to_day', 'topic_huwelijk', 'topic_home', 
                         'topic_leisure', 'topic_werk', 'topic_short', 'topic_sweet']]

# Bereken de som van elk onderwerp per auteur
author_topic_sums = demographic.groupby('author').sum()

# Maak een horizontale barchart voor elk onderwerp met author 1 links en author 2 rechts
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12), sharex=True)

# Plot voor author 1
author_topic_sums.loc['sudsy-mole'].plot(kind='barh', ax=axes[0], color='blue')
axes[0].set_title('Author 1')
axes[0].set_xlabel('Sum')
axes[0].set_ylabel('Topics')

# Plot voor author 2
author_topic_sums.loc['kaleidoscopic-ferret'].plot(kind='barh', ax=axes[1], color='red')
axes[1].set_title('Author 2')
axes[1].set_xlabel('Sum')
axes[1].set_ylabel('Topics')

plt.tight_layout()
plt.show()

Inladen van de Player Role json

In [ ]:
player_roles = pd.read_json(role_file, encoding = "latin")
player_roles.head()

Merge dataframes

In [ ]:
merged_df = pd.merge(merged_df, player_roles, left_on='author', right_on='Author')
merged_df["has_image"] = merged_df["message"].str.contains("<Media weggela").astype(int)
merged_df["has_tikkie"] = merged_df["message"].str.contains("https://tikkie.me").astype(int)
merged_df.shape

In [ ]:
q1 = lambda x: np.quantile(x, 0.25)
q3 = lambda x: np.quantile(x, 0.75)

merged_df["hour"] = merged_df.timestamp.dt.hour
quantiles = merged_df.groupby("hour")["message_length"].agg([q1, q3])
quantiles.columns = ["q1", "q3"]

p = quantiles.reindex(range(24), fill_value=0)

sns.scatterplot(data=p, x="q1", y=p.index, color="grey")
sns.scatterplot(data=p, x="q3", y=p.index, color="grey")
for index, row in p.iterrows():
    sns.lineplot(x=[row["q1"], row["q3"]], y=[index, index], color="grey")

In [ ]:
tikkie = merged_df[merged_df["has_tikkie"] == 1]
tikkie.head()
tikkie.shape


Aantal berichten per maand visualiseren.

In [ ]:
merged_df_2022 = merged_df[merged_df["timestamp"].dt.year == 2022]
mask = (merged_df["timestamp"] >= "2021-06-01") & (merged_df["timestamp"] <= "2022-09-01")
filtered_df = merged_df.loc[mask]
filtered_emoji_df = filtered_df[filtered_df["has_emoji"] == True]
filtered_images_df = filtered_df[filtered_df["has_image"] == 1]

messages_per_month = filtered_df.groupby(filtered_df["timestamp"].dt.to_period("M")).size()
messages_per_week = filtered_df.groupby(filtered_df["timestamp"].dt.to_period("W")).size()
messages_per_day = filtered_df.groupby(filtered_df["timestamp"].dt.to_period("D")).size()
emojis_per_month = filtered_emoji_df.groupby(filtered_df["timestamp"].dt.to_period("M")).size()
emojis_per_week = filtered_emoji_df.groupby(filtered_df["timestamp"].dt.to_period("W")).size()
emojis_per_day = filtered_emoji_df.groupby(filtered_df["timestamp"].dt.to_period("D")).size()
images_per_month = filtered_images_df.groupby(filtered_df["timestamp"].dt.to_period("M")).size()
images_per_week = filtered_images_df.groupby(filtered_df["timestamp"].dt.to_period("W")).size()
images_per_day = filtered_images_df.groupby(filtered_df["timestamp"].dt.to_period("D")).size()


plt.figure(figsize=(20, 6))
plt.figure(figsize=(20, 6))
ax = plt.gca()

# Plot de berichten per week en per maand
messages_per_week.plot(kind="line", ax=ax)
images_per_week.plot(kind="line", ax=ax)

# Voeg de grijze achtergrond toe voor februari 2022 en juni 2022
ax.axvspan("2022-02-13", "2022-02-21", color="blue", alpha=0.3)
ax.axvspan("2022-05-28","2022-06-10", color="grey", alpha=0.3)

# messages_per_week.plot(kind="line")
# images_per_week.plot(kind="line")

highlight_date = pd.Timestamp("2022-06-02")
highlight_value = messages_per_week.loc[highlight_date.to_period("W")]
ax.annotate('Play Offs', xy=(highlight_date, highlight_value), xytext=(highlight_date, highlight_value + 80),
            arrowprops=dict(facecolor='black', shrink=0.05))

highlight_date = pd.Timestamp("2022-02-19")
highlight_value = messages_per_week.loc[highlight_date.to_period("W")]
ax.annotate('Teamweekend', xy=(highlight_date, highlight_value), xytext=(highlight_date, highlight_value + 80),
            arrowprops=dict(facecolor='black', shrink=0.005))

plt.title("Aantal berichten per maand")
plt.xlabel("Maand")
plt.ylabel("Aantal berichten")
plt.show()

Voeg verschillende kolommen toe aan de data, met name kolommen gerelateerd aan datetime, maar ook een kolom die kijkt of er media is gestuurd met het bericht.

In [ ]:
merged_df["day_of_month"] = merged_df["timestamp"].dt.day
merged_df["day"] = merged_df["timestamp"].dt.day_name()
merged_df["month_number"] = merged_df["timestamp"].dt.month
merged_df["month_name"] = merged_df["timestamp"].dt.month_name()
merged_df["year"] = merged_df["timestamp"].dt.year
merged_df["has_image"] = merged_df["message"].str.contains("<Media weggelaten>").astype(int)
merged_df.head()

Maak visualisatie van de gemiddelde lengte van de berichten afhangende van de functie

In [ ]:
merged_df["message_length"] = merged_df["message"].str.len()
player_message_count = merged_df[merged_df['Function'] == 'Player'].count()['message']
staff_message_count = merged_df[merged_df['Function'] == 'Staff'].count()['message']

p1 = (
    merged_df[["Function", "message_length"]]
    .groupby("Function")
    .mean()
    .sort_values("message_length", ascending=False)
)

sns.barplot(x=p1.index, y=p1["message_length"], palette = ["red", "lightgrey"]  )
for i, v in enumerate(p1["message_length"]):
    plt.text(i, v*0.98, f'{v:.1f}', ha='center', va='top', fontsize=12)
plt.xlabel("Function within team")
plt.ylabel("Average Message length")
plt.title("Staff members sending longer messages")


plt.figtext(0.05, -0.05, f"Gebaseerd op {player_message_count:,}".replace(',', '.') + f" berichten van de players en {staff_message_count:,}".replace(',', '.') +f" berichten van de staff.", 
            ha='left', va='center', fontsize=8, fontstyle= "italic")

plt.tight_layout() 
plt.subplots_adjust(bottom=0.1) 

Show number of messages per year.

In [ ]:
no_of_messages = (
    merged_df[["Function", "message", "has_image"]]
    .groupby("Function")
    .agg(
        no_of_messages=("message","count"),
        no_of_images=("has_image", "sum")
    )
    .sort_values("Function", ascending=True)
)
no_of_messages[ "percentage_with_images"] = (no_of_messages['no_of_images'] / no_of_messages['no_of_messages'] * 100)

no_of_messages

Count the number of messages per day and group them.

In [ ]:
month_day_count = merged_df.groupby(["month_name", "month_number", "day_of_month"]).size().unstack(fill_value = 0)
year_month_day_count = merged_df.groupby(["year", "month_name", "month_number", "day_of_month"]).size().unstack(fill_value = 0)
month_day_count.head()
year_month_day_count.head()
# average_per_day = month_day_count.mean(axis=0)

In [ ]:
years = year_month_day_count.index.get_level_values("year").unique()

for year in years:
    data_for_year = year_month_day_count.loc[year]
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(data_for_year, cmap="YlGnBu", annot=True, fmt="d")
    plt.title(f"Heatmap van tellingen voor jaar {year}")
    plt.xlabel("Dag van de maand")
    plt.ylabel("Maand")
    plt.show()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(month_day_count, annot=True, fmt="d", linewidths=0.5, cmap="GnBu")
plt.xticks(
    ticks=range(31),
    labels=range(1, 32)
)

plt.yticks(
    ticks=range(12),
    labels=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    rotation=0
)

plt.title("Heatmap van verzonden berichten per dag per maand")
plt.xlabel("Dag van de maand")
plt.ylabel("Maand")
plt.show()